# 파일 가져오기

In [ ]:
import os

# wav_file 폴더 경로 설정
wav_folder = './wav_file'

# all_files변수에 wav_folder경로에 있는 파일 리스트로 저장
all_files = os.listdir(wav_folder)

wav_files = []
# 가져온 파일 목록에서 각 파일 하나씩 확인
for f in all_files:
    if f.endswith('.WAV'):
        # 조건에 맞는 파일들을 리스트에 추가
        wav_files.append(f)
print(wav_files)

# 파일 시각화 및 소리 재생

In [ ]:
from IPython.display import Audio


# 각 파일에 대해 시각화 및 음성 재생
for wav_file in wav_files:
    file_path = os.path.join(wav_folder, wav_file)
    
    # 음성 파일 로드
    y, sr = librosa.load(file_path, sr=None)  # 원래 샘플링 레이트 유지
    
    # 파형 시각화
    plt.figure(figsize=(10, 4))
    librosa.display.waveshow(y, sr=sr)
    plt.title(f"Waveform of {wav_file}")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.show()
    
    # 음성 재생
    print(f"Playing {wav_file}:")
    display(Audio(data=y, rate=sr))

원본 파일 형태를 시각화해서 보면 클릭음과 기계음이 들어간 걸 확인

# 데이터 전처리

### 노이즈 제거 1 : 클릭음 제거

In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import write
from IPython.display import display

# 출력 폴더 설정
output_folder = './noise1'
os.makedirs(output_folder, exist_ok=True)

# STFT 계산 및 처리 (코드가 정상적으로 들여쓰기됨)
def process_audio(y, sr, wav_file):
    # STFT 계산
    D = librosa.stft(y)

    # 주파수 범위 설정 (1300 Hz ~ 5000 Hz)
    freqs = librosa.fft_frequencies(sr=sr)
    min_freq = 2100
    max_freq = 2300
    freq_mask = np.logical_and(freqs >= min_freq, freqs <= max_freq)

    # freq_mask를 2D 배열로 확장 (시간 프레임마다 동일하게 적용)
    freq_mask_2d = np.repeat(freq_mask[:, np.newaxis], D.shape[1], axis=1)

    # 하모닉과 퍼커시브 분리
    harmonic, percussive = librosa.decompose.hpss(D)

    # 퍼커시브 부분에서 지정된 주파수 범위만 남기고 나머지는 제거
    percussive_filtered = percussive * freq_mask_2d

    # 퍼커시브 제거 후 결과
    y_harmonic = librosa.istft(harmonic + percussive_filtered)

    # 결과 저장 경로 설정
    output_path = os.path.join(output_folder, f"cleaned_{wav_file}")
    write(output_path, sr, (y_harmonic * 32767).astype(np.int16))  # 16비트 PCM 형식으로 저장

    # 신호 시각화
    time = np.linspace(0, len(y) / sr, len(y))
    plt.figure(figsize=(10, 4))
    plt.plot(time, y, label="Original Signal", alpha=0.6)
    plt.plot(time[:len(y_harmonic)], y_harmonic, label="Signal without Noise", color='orange', alpha=0.8)
    plt.title(f"Signal Before and After Noise Removal ({wav_file})")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.legend()
    plt.grid()
    plt.show()

    # 원본 및 제거된 신호 재생
    print(f"Playing original audio for {wav_file}:")
    display(Audio(data=y, rate=sr))

    print(f"Playing cleaned audio for {wav_file}:")
    display(Audio(filename=output_path))

# 파일 경로 및 처리
wav_folder = './wav_file'
wav_files = [f for f in os.listdir(wav_folder) if f.endswith('.WAV')]

# 각 파일에 대해 함수 호출
for wav_file in wav_files:
    file_path = os.path.join(wav_folder, wav_file)
    
    # 오디오 파일 로드
    y, sr = librosa.load(file_path, sr=None)
    
    # 함수 호출
    process_audio(y, sr, wav_file)


- > 문제 발생 : 노이즈가 제거가 되었으나 소리가 너무 작아짐

### 노이즈 제거된 파일 증폭

In [ ]:
# 입력 및 출력 폴더 설정
input_folder = './noise1'  # 노이즈 제거된 파일이 저장된 폴더
output_amplified_folder = './amplified_noise1'

# 출력 폴더 생성
os.makedirs(output_amplified_folder, exist_ok=True)

# 증폭 계수 (소리를 키우는 정도)
amplification_factor = 2.0  # 2배로 증폭

# 디버깅: 입력 폴더와 파일 확인
print(f"Input folder: {input_folder}")
print(f"Output folder: {output_amplified_folder}")

# input_folder의 파일 가져오기
amplified_files = [f for f in os.listdir(input_folder) if f.endswith('.WAV')]
print(f"Files found: {amplified_files}")


### 증폭된 파일 시각화

In [ ]:
# 증폭된 파일이 저장된 폴더 설정
output_amplified_folder = './amplified_noise1'

# 증폭된 파일 가져오기
amplified_files = [f for f in os.listdir(output_amplified_folder) if f.endswith('.WAV')]
print(f"Amplified files found: {amplified_files}")

# 파일이 없는 경우 처리
if not amplified_files:
    print("No amplified .wav files found in the output folder!")
else:
    for file_name in amplified_files:
        file_path = os.path.join(output_amplified_folder, file_name)
        
        # 음성 파일 로드
        y, sr = librosa.load(file_path, sr=None)  # 원래 샘플링 레이트 유지
        
        # 파형 시각화
        plt.figure(figsize=(10, 4))
        librosa.display.waveshow(y, sr=sr)
        plt.title(f"Waveform of {file_name}")
        plt.xlabel("Time (s)")
        plt.ylabel("Amplitude")
        plt.show()
        
        # 음성 재생
        print(f"Playing amplified audio: {file_name}")
        display(Audio(data=y, rate=sr))


좋았던 점 : 2배 소리를 증폭 시켰더니 클릭음이 제거가 안된 부분의 소리를 들을 수 있어 주파수 대역을 타이트하게 잡을 수 있었음

### 노이즈 제거 2: 기계음 제거 

- 이 코드는 noisereduce 라이브러리를 사용하여 오디오 신호에서 시간 및 주파수 마스크를 기반으로 기계음을 감지하고, 감지된 잡음 성분을 감소시키는 방식으로 기계음을 제거

In [ ]:
# 입력 및 출력 폴더 설정
input_folder = './amplified_noise1'  # 증폭된 파일이 저장된 폴더
output_filtered_folder = './filtered_noise2'  # 필터링된 파일 저장 폴더

# 출력 폴더 생성
os.makedirs(output_filtered_folder, exist_ok=True)

# input_folder의 파일 가져오기
amplified_files = [f for f in os.listdir(input_folder) if f.endswith('.WAV')]
print(f"Files found: {amplified_files}")

# 파일이 없는 경우 처리
if not amplified_files:
    print("No .wav files found in the input folder!")
else:
    for file_name in amplified_files:
        input_path = os.path.join(input_folder, file_name)
        output_path = os.path.join(output_filtered_folder, f"filtered_{file_name}")
        
        # WAV 파일 불러오기
        audio = AudioSegment.from_wav(input_path)

        # 오디오 데이터를 numpy 배열로 변환
        audio_data = np.array(audio.get_array_of_samples())

        # 잡음 제거
        reduced_audio_data = reduce_noise(
            y=audio_data,
            sr=audio.frame_rate,
            prop_decrease=1,  # 잡음 감소 비율
            time_mask_smooth_ms=4000,  # 시간 마스크 크기 (ms)
            freq_mask_smooth_hz=400,  # 주파수 마스크 크기 (Hz)
        )

        # 처리된 오디오 데이터를 AudioSegment로 변환
        reduced_audio = AudioSegment(
            reduced_audio_data.tobytes(),
            frame_rate=audio.frame_rate,
            sample_width=audio.sample_width,
            channels=audio.channels
        )

        # 필터링된 오디오 저장
        reduced_audio.export(output_path, format="wav")
        print(f"Filtered and saved: {output_path}")

        # 오디오 재생 (잡음이 줄어든 파일)
        print(f"Playing filtered audio for {file_name}:")
        display(Audio(output_path))

        # 파형 시각화
        plt.figure(figsize=(12, 6))

        # 원본 오디오 파형
        plt.subplot(2, 1, 1)
        plt.plot(np.arange(len(audio_data)) / audio.frame_rate, audio_data)
        plt.title(f'Original Audio Waveform: {file_name}')
        plt.xlabel('Time [s]')
        plt.ylabel('Amplitude')

        # 필터링된 오디오 파형
        plt.subplot(2, 1, 2)
        plt.plot(np.arange(len(reduced_audio_data)) / audio.frame_rate, reduced_audio_data)
        plt.title(f'Filtered Audio Waveform: {file_name}')
        plt.xlabel('Time [s]')
        plt.ylabel('Amplitude')

        plt.tight_layout()
        plt.show()


문제 발생
- 1. 기계음을 제거하면 음성 파일의 소리가 줄어들며, 소리가 없는 부분의 파형이 0으로 수렴하는 현상이 확인
  2. 소리가 있는 부분의 기걔음 제거가 안되는 걸 확인

### 노이즈 제거된 파일 증폭 및 시각화

In [ ]:
# 입력 및 출력 폴더 설정
input_folder = './filtered_noise2'  # 노이즈 제거된 파일이 저장된 폴더
output_amplified_folder = './amplified_noise2'

# 출력 폴더 생성
os.makedirs(output_amplified_folder, exist_ok=True)

# 증폭 계수 (소리를 키우는 정도)
amplification_factor = 2.0  # 2배로 증폭

# 디버깅: 입력 폴더와 파일 확인
print(f"Input folder: {input_folder}")
print(f"Output folder: {output_amplified_folder}")

# input_folder의 파일 가져오기
input_files = [f for f in os.listdir(input_folder) if f.lower().endswith('.wav')]
print(f"Files found: {input_files}")

# 파일이 없는 경우 처리
if not input_files:
    print("No .wav files found in the input folder!")
else:
    for file_name in input_files:
        input_path = os.path.join(input_folder, file_name)
        
        # 오디오 파일 로드
        print(f"Processing file: {input_path}")
        y, sr = librosa.load(input_path, sr=None)
        
        # 볼륨 증폭
        y_amplified = y * amplification_factor
        
        # 클리핑 방지 (샘플 값이 -1.0 ~ 1.0 범위를 초과하지 않도록 조정)
        y_amplified = np.clip(y_amplified, -1.0, 1.0)
        
        # 증폭된 파일 저장 경로
        output_path = os.path.join(output_amplified_folder, f"amplified_{file_name}")
        write(output_path, sr, (y_amplified * 32767).astype(np.int16))  # 16비트 PCM 형식으로 저장
        
        # 결과 출력
        print(f"Amplified and saved: {output_path}")
        
        # 증폭된 오디오 재생
        print(f"Playing amplified audio for {file_name}:")
        display(Audio(data=y_amplified, rate=sr))

        # 파형 시각화
        plt.figure(figsize=(10, 4))
        librosa.display.waveshow(y, sr=sr)
        plt.title(f"Waveform of {file_name}")
        plt.xlabel("Time (s)")
        plt.ylabel("Amplitude")
        plt.show()


문제 발생 : 소리는 커졌지만 여전히 음성 파형이 있는 곳의 기계음은 제거 되지 않았다